In [8]:
from tinydb import TinyDB, Query
from sqlmodel import SQLModel, Field
from typing import Optional
from abc import ABC
from datetime import datetime, timezone

# Assuming BModel is already defined
class BModel(ABC, SQLModel):
    id: int | None = Field(default=None, primary_key=True)
    type: Optional[str] = Field(default=None, description="Type of the record/object")
    name: Optional[str] = Field(description="Name of the item")
    description: Optional[str] = Field(default=None, description="Description of the item")
    created_at: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    updated_at: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    is_active: bool = True

    class Config:
        arbitrary_types_allowed = True  # Allows for more flexible type definitions

    def validate(self) -> None:
        if not self.name:
            raise ValueError("Name cannot be empty")


# Step 1: Connect to TinyDB and fetch schema
tinydb = TinyDB('db1_schema.json')
schema_table = tinydb.table('schema_versions')
# print (schema_table)

# Fetch the latest schema version
def get_latest_schema():
    latest_version = max([entry['version'] for entry in schema_table.all()])
    return schema_table.search(Query().version == latest_version)[0]['schema']

# Step 2: Generate SQLModels from schema
def create_sqlmodel_classes(schema):
    models = {}
    
    for table_name, columns in schema.items():
        # Dynamically build the class with fields
        fields = {}
        for column in columns:
            column_name = column['name']
            column_type = column['type'].upper()
            
            # Map SQLite column types to Python/SQLModel types
            if "INT" in column_type:
                field_type = int
            elif "TEXT" in column_type:
                field_type = str
            elif "REAL" in column_type:
                field_type = float
            elif "BLOB" in column_type:
                field_type = bytes
            else:
                field_type = str  # default to string if unknown type
            
            # Add field to the model
            fields[column_name] = (Optional[field_type], Field(default=None))

        # Create a new class for the table and inherit from BModel
        model_class = type(table_name, (BModel,), fields)
        models[table_name] = model_class
    
    return models

# Step 3: Generate models
latest_schema = get_latest_schema()
sqlmodels = create_sqlmodel_classes(latest_schema)

# Step 4: Print or use generated models
# for model_name, model_class in sqlmodels.items():
#     print(f"Generated model: {model_name}")
#     print(model_class.schema_json(indent=2))  # This will print the schema in JSON format


PydanticUserError: A non-annotated attribute was detected: `tenant_id = (typing.Optional[int], FieldInfo(annotation=NoneType, required=False, default=None))`. All model fields require a type annotation; if `tenant_id` is not meant to be a field, you may be able to resolve this error by annotating it as a `ClassVar` or updating `model_config['ignored_types']`.

For further information visit https://errors.pydantic.dev/2.9/u/model-field-missing-annotation

In [9]:
from tinydb import TinyDB, Query
from sqlmodel import SQLModel, Field
from typing import Optional
from abc import ABC
from datetime import datetime, timezone

# Assuming BModel is already defined
class BModel(ABC, SQLModel):
    id: int | None = Field(default=None, primary_key=True)
    type: Optional[str] = Field(default=None, description="Type of the record/object")
    name: Optional[str] = Field(description="Name of the item")
    description: Optional[str] = Field(default=None, description="Description of the item")
    created_at: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    updated_at: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    is_active: bool = True

    class Config:
        arbitrary_types_allowed = True  # Allows for more flexible type definitions

    def validate(self) -> None:
        if not self.name:
            raise ValueError("Name cannot be empty")


# Step 1: Connect to TinyDB and fetch schema
tinydb = TinyDB('db_schema.json')
schema_table = tinydb.table('schema_versions')

# Fetch the latest schema version
def get_latest_schema():
    latest_version = max([entry['version'] for entry in schema_table.all()])
    return schema_table.search(Query().version == latest_version)[0]['schema']

# Step 2: Generate SQLModels from schema
def create_sqlmodel_classes(schema):
    models = {}
    
    for table_name, columns in schema.items():
        # Prepare a dictionary for storing class attributes
        class_attributes = {
            '__annotations__': {}
        }

        for column in columns:
            column_name = column['name']
            column_type = column['type'].upper()
            
            # Map SQLite column types to Python/SQLModel types
            if "INT" in column_type:
                field_type = int
            elif "TEXT" in column_type:
                field_type = str
            elif "REAL" in column_type:
                field_type = float
            elif "BLOB" in column_type:
                field_type = bytes
            else:
                field_type = str  # default to string if unknown type
            
            # Add the field annotation and default value
            class_attributes['__annotations__'][column_name] = Optional[field_type]
            class_attributes[column_name] = Field(default=None)
        
        # Create the new model class dynamically
        model_class = type(table_name, (BModel,), class_attributes)
        models[table_name] = model_class
    
    return models

# Step 3: Generate models
latest_schema = get_latest_schema()
sqlmodels = create_sqlmodel_classes(latest_schema)

# Step 4: Print or use generated models
for model_name, model_class in sqlmodels.items():
    print(f"Generated model: {model_name}")
    print(model_class.schema_json(indent=2))  # This will print the schema in JSON format


ValueError: max() iterable argument is empty

In [12]:
from tinydb import TinyDB, Query
from sqlmodel import SQLModel, Field
from typing import Optional
from abc import ABC
from datetime import datetime, timezone

# Assuming BModel is already defined
class BModel(ABC, SQLModel):
    id: int | None = Field(default=None, primary_key=True)
    type: Optional[str] = Field(default=None, description="Type of the record/object")
    name: Optional[str] = Field(description="Name of the item")
    description: Optional[str] = Field(default=None, description="Description of the item")
    created_at: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    updated_at: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    is_active: bool = True

    class Config:
        arbitrary_types_allowed = True  # Allows for more flexible type definitions

    def validate(self) -> None:
        if not self.name:
            raise ValueError("Name cannot be empty")


# Step 1: Connect to TinyDB and fetch schema
tinydb = TinyDB('db1_schema.json')
schema_table = tinydb.table('schema_versions')

# Fetch the latest schema version
def get_latest_schema():
    if not schema_table:  # Check if schema_table is empty
        raise ValueError("No schema versions found in TinyDB!")
    
    latest_version = max([entry['version'] for entry in schema_table.all()])
    return schema_table.search(Query().version == latest_version)[0]['schema']

# Step 2: Generate SQLModels from schema
def create_sqlmodel_classes(schema):
    models = {}
    
    for table_name, columns in schema.items():
        # Prepare a dictionary for storing class attributes
        class_attributes = {
            '__annotations__': {}
        }

        for column in columns:
            column_name = column['name']
            column_type = column['type'].upper()
            
            # Map SQLite column types to Python/SQLModel types
            if "INT" in column_type:
                field_type = int
            elif "TEXT" in column_type:
                field_type = str
            elif "REAL" in column_type:
                field_type = float
            elif "BLOB" in column_type:
                field_type = bytes
            else:
                field_type = str  # default to string if unknown type
            
            # Add the field annotation and default value
            class_attributes['__annotations__'][column_name] = Optional[field_type]
            class_attributes[column_name] = Field(default=None)
        
        # Create the new model class dynamically
        model_class = type(table_name, (BModel,), class_attributes)
        models[table_name] = model_class
    
    return models

# Step 3: Generate models
try:
    latest_schema = get_latest_schema()
    sqlmodels = create_sqlmodel_classes(latest_schema)

    # Step 4: Print or use generated models
    for model_name, model_class in sqlmodels.items():
        print(f"Generated model: {model_name}")
        print(model_class.schema_json(indent=2))  # This will print the schema in JSON format
except ValueError as e:
    print(e)


Generated model: Tenant
{
  "properties": {
    "id": {
      "anyOf": [
        {
          "type": "integer"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "title": "Id"
    },
    "type": {
      "anyOf": [
        {
          "type": "string"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "description": "Type of the record/object",
      "title": "Type"
    },
    "name": {
      "anyOf": [
        {
          "type": "string"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "title": "Name"
    },
    "description": {
      "anyOf": [
        {
          "type": "string"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "description": "Description of the item",
      "title": "Description"
    },
    "created_at": {
      "format": "date-time",
      "title": "Created At",
      "type": "string"
    },

In [14]:
from tinydb import TinyDB, Query
from sqlmodel import SQLModel, Field
from typing import Optional, Type
from abc import ABC
from datetime import datetime, timezone

# Step 1: Define the BModel as the base class
class BModel(ABC, SQLModel):
    id: int | None = Field(default=None, primary_key=True)
    type: Optional[str] = Field(default=None, description="Type of the record/object")
    name: Optional[str] = Field(description="Name of the item")
    description: Optional[str] = Field(default=None, description="Description of the item")
    created_at: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    updated_at: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    is_active: bool = True

    class Config:
        arbitrary_types_allowed = True  # Allows for more flexible type definitions

    def validate(self) -> None:
        if not self.name:
            raise ValueError("Name cannot be empty")


# Step 2: Connect to TinyDB and fetch the latest schema
tinydb = TinyDB('db1_schema.json')
schema_table = tinydb.table('schema_versions')

def get_latest_schema():
    """Retrieve the latest schema version from TinyDB."""
    if not schema_table:
        raise ValueError("No schema versions found in TinyDB!")
    
    latest_version = max([entry['version'] for entry in schema_table.all()])
    return schema_table.search(Query().version == latest_version)[0]['schema']


# Step 3: Generate SQLModels from schema
def create_sqlmodel_classes(schema: dict) -> dict[str, Type[SQLModel]]:
    """Create SQLModel classes dynamically from the schema."""
    models = {}

    for table_name, columns in schema.items():
        # Prepare the dictionary for storing class attributes
        class_attributes = {
            '__annotations__': {}  # To hold the field annotations
        }

        for column in columns:
            column_name = column['name']
            column_type = column['type'].upper()
            
            # Map SQLite types to Python/SQLModel types
            if "INT" in column_type:
                field_type = int
            elif "TEXT" in column_type:
                field_type = str
            elif "REAL" in column_type:
                field_type = float
            elif "BLOB" in column_type:
                field_type = bytes
            else:
                field_type = str  # Default to string if type is unknown

            # Add the field annotation and default value
            class_attributes['__annotations__'][column_name] = Optional[field_type]
            class_attributes[column_name] = Field(default=None)

        # Dynamically create the SQLModel class and inherit from BModel
        model_class = type(table_name, (BModel,), class_attributes)
        models[table_name] = model_class

    return models


# Step 4: Generate the models and print them
try:
    latest_schema = get_latest_schema()
    sqlmodels = create_sqlmodel_classes(latest_schema)

    # Print the generated models
    for model_name, model_class in sqlmodels.items():
        print(f"Generated model: {model_name}")
        print(model_class.schema_json(indent=2))  # Print the SQLModel schema in JSON format

except ValueError as e:
    print(e)


Generated model: Tenant
{
  "properties": {
    "id": {
      "anyOf": [
        {
          "type": "integer"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "title": "Id"
    },
    "type": {
      "anyOf": [
        {
          "type": "string"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "description": "Type of the record/object",
      "title": "Type"
    },
    "name": {
      "anyOf": [
        {
          "type": "string"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "title": "Name"
    },
    "description": {
      "anyOf": [
        {
          "type": "string"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "description": "Description of the item",
      "title": "Description"
    },
    "created_at": {
      "format": "date-time",
      "title": "Created At",
      "type": "string"
    },

In [16]:
import os
from tinydb import TinyDB, Query
from sqlmodel import SQLModel, Field
from typing import Optional
from abc import ABC
from datetime import datetime, timezone

# Step 1: Define the BModel as the base class
class BModel(ABC, SQLModel):
    id: int | None = Field(default=None, primary_key=True)
    type: Optional[str] = Field(default=None, description="Type of the record/object")
    name: Optional[str] = Field(description="Name of the item")
    description: Optional[str] = Field(default=None, description="Description of the item")
    created_at: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    updated_at: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    is_active: bool = True

    class Config:
        arbitrary_types_allowed = True  # Allows for more flexible type definitions

    def validate(self) -> None:
        if not self.name:
            raise ValueError("Name cannot be empty")


# Step 2: Connect to TinyDB and fetch the latest schema
tinydb = TinyDB('db1_schema.json')
schema_table = tinydb.table('schema_versions')

def get_latest_schema():
    """Retrieve the latest schema version from TinyDB."""
    if not schema_table:
        raise ValueError("No schema versions found in TinyDB!")
    
    latest_version = max([entry['version'] for entry in schema_table.all()])
    return schema_table.search(Query().version == latest_version)[0]['schema']


# Step 3: Generate SQLModel class definitions as Python code
def generate_sqlmodel_class_code(table_name: str, columns: list) -> str:
    """Generate Python code for a SQLModel class."""
    class_code = f"from sqlmodel import SQLModel, Field\nfrom typing import Optional\nfrom datetime import datetime, timezone\n\n"
    class_code += f"class {table_name}(BModel):\n"

    for column in columns:
        column_name = column['name']
        column_type = column['type'].upper()
        
        # Map SQLite types to Python/SQLModel types
        if "INT" in column_type:
            field_type = "int"
        elif "TEXT" in column_type:
            field_type = "str"
        elif "REAL" in column_type:
            field_type = "float"
        elif "BLOB" in column_type:
            field_type = "bytes"
        else:
            field_type = "str"  # default to string if unknown type
        
        class_code += f"    {column_name}: Optional[{field_type}] = Field(default=None)\n"

    return class_code


# Step 4: Save the generated classes to Python files
def save_model_files(models: dict, output_dir: str):
    """Save each model as a Python file."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for table_name, model_code in models.items():
        file_path = os.path.join(output_dir, f"{table_name.lower()}.py")
        with open(file_path, "w") as file:
            file.write(model_code)
        print(f"Model file created: {file_path}")


# Step 5: Main workflow
try:
    latest_schema = get_latest_schema()

    # Directory where Python model files will be saved
    output_directory = "models"

    # Generate model code for each table
    models = {}
    for table_name, columns in latest_schema.items():
        models[table_name] = generate_sqlmodel_class_code(table_name, columns)
    
    # Save the models as Python files
    save_model_files(models, output_directory)

except ValueError as e:
    print(e)


Model file created: models/tenant.py
Model file created: models/sqlite_sequence.py
Model file created: models/category.py
Model file created: models/product.py
